# Initialize data-model comparisons

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import astropy.units as u
%config InlineBackend.figure_format = 'retina'

In [ ]:
import numpy as np

In [ ]:
df = pd.read_csv('../../sf/luhman16A_btjd_2285p65/m110/output/marley_grid/run01/spec_config.csv')

In [ ]:
df.head()

In [ ]:
plt.rcParams['figure.facecolor'] = 'white'

In [ ]:
plt.step(df.wl, df.data, label='IGRINS $m=110$')
plt.step(df.wl, df.model_composite, label='Sonora cloud-free \n$T=1350$ K, $\log{g}=4.5$')
plt.legend();
plt.xlabel('$\lambda \; (\AA)$')
plt.ylabel('$\propto$ Flux')
plt.ylim(0)
plt.title('Luhman 16 quicklook')
plt.savefig('../../figures/Luhman16_quicklook.png', dpi=300, bbox_inches='tight')

## Add a cloudy mixture model component

In [ ]:
morley_clouds_fns = sorted(glob.glob('/home/gully/libraries/raw/morley_clouds_20210322/ldwarf_specs/*.spec'))

In [ ]:
len(morley_clouds_fns)

In [ ]:
fn = morley_clouds_fns[5]

In [ ]:
fn

In [ ]:
! ls /home/gully/libraries/raw/morley_clouds_20210322/ldwarf_specs/

In [ ]:
! head -n 4 /home/gully/libraries/raw/morley_clouds_20210322/ldwarf_specs/t1600g316f2_m0.0_co1.0.spec

In [ ]:
! head -n 4 /home/gully/libraries/raw/marley/sp_t1000g178nc_m0.0

Hmmm the *units* differ, but are still compatible.  Let's use astropy to convert them to the same unit.

In [ ]:
df_model = pd.read_csv(fn, skiprows=[0,1], delim_whitespace=True, names=['wl_um','flux'])
df_model = df_model.sort_values('wl_um').reset_index(drop=True)

In [ ]:
df_nc = pd.read_csv('/home/gully/libraries/raw/marley/sp_t1300g178nc_m0.0', skiprows=[0,1], delim_whitespace=True, names=['wl_um','flux'])
df_nc = df_nc.sort_values('wl_um').reset_index(drop=True)

In [ ]:
marley_unit = u.erg/u.cm**2/u.s/u.Hz
morley_unit = u.Watt/u.m**2/u.m

morley_flux_w_units = (df_model.flux.values*morley_unit)
morley_wavelength = df_model.wl_um.values*u.micron

morley_cgs = morley_flux_w_units.to(marley_unit, equivalencies=u.spectral_density(morley_wavelength))

In [ ]:
df_model['flux_cgs'] = morley_cgs
df_nc['flux_cgs'] = df_nc.flux

In [ ]:
def trim_to_tess_and_IGRINS(df_model):
    '''Trim a dataframe to only TESS and IGRINS bandpass'''
    tess_mask = (df_model.wl_um > 0.6) & (df_model.wl_um < 1.050)
    igrins_mask = (df_model.wl_um > 1.45) & (df_model.wl_um < 2.55)
    df_model = df_model[tess_mask | igrins_mask].reset_index(drop=True)
    return df_model

In [ ]:
df_clouds = trim_to_tess_and_IGRINS(df_model)

In [ ]:
df_nc = trim_to_tess_and_IGRINS(df_nc)

In [ ]:
plt.step(df_clouds.wl_um, df_clouds.flux_cgs, label='Cloudy')
plt.step(df_nc.wl_um, df_nc.flux_cgs, label='Cloud-free')
plt.title(r'$T_{\mathrm{eff}}$ = 1000 K, $g=178$ m/s$^2$')
plt.yscale('log')
plt.xscale('log')
plt.legend(loc='best');

Awesome!  That looks right!  On average, the clouds are brighter than the cloud-free atmosphere, for the same sized patch.

Let's overplot for just the range of the above IGRINS order.

In [ ]:
plt.step(df.wl, df.data, label='IGRINS $m=110$')
plt.step(df.wl, df.model_composite, label='Sonora cloud-free \n$T=1350$ K, $\log{g}=4.5$, $v\sin{i}=\;30\;$km/s')
plt.legend();
plt.xlabel('$\lambda \; (\AA)$')
plt.ylabel('$\propto$ Flux')
plt.ylim(0)
plt.title('Luhman 16 quicklook')
plt.savefig('../../figures/Luhman16_quicklook.png', dpi=300, bbox_inches='tight')

In [ ]:
from scipy.ndimage import gaussian_filter1d

In [ ]:
gaussian_filter1d(df_nc.flux_cgs, sigma=25)

In [ ]:
plt.figure(figsize=(15, 6))
plt.step(df.wl/10000, df.data*1e-6, label='IGRINS $m=110$', color='k')

smoothed_cloudy = gaussian_filter1d(df_clouds.flux_cgs, sigma=2)
smoothed_nocloud = gaussian_filter1d(df_nc.flux_cgs, sigma=11)
ff = 0.4
#net = ff*smoothed_cloudy + (1-ff)*smoothed_nocloud

#plt.step(df_nc.wl_um, net, label='Composite ($f=40$)')

plt.step(df_nc.wl_um, (1-ff)*smoothed_nocloud, label='Cloud-free')
plt.step(df_clouds.wl_um, ff*smoothed_cloudy, label='Cloudy')
plt.title(r'$T_{\mathrm{eff}}$ = 1000 K, $g=178$ m/s$^2$')
plt.legend(loc='best');
plt.xlim(1.6220, 1.64000)

In [ ]:
from muler.igrins import IGRINSSpectrum

In [ ]:
reduced_fns = glob.glob('../../data/IGRINS/originals/GS-2021A-DD-104/*/reduced/SDCK*.spec_a0v.fits')

In [ ]:
spec1 = IGRINSSpectrum(file=reduced_fns[1], order=10).normalize()

In [ ]:
spec1

In [ ]:
ax = spec1.plot()
plt.figure(figsize=(15, 6))
#plt.step(df.wl/10000, df.data*1e-6, label='IGRINS $m=110$', color='k')

smoothed_cloudy = gaussian_filter1d(df_clouds.flux_cgs, sigma=2)
smoothed_nocloud = gaussian_filter1d(df_nc.flux_cgs, sigma=12)
ff = 0.4
#net = ff*smoothed_cloudy + (1-ff)*smoothed_nocloud

#plt.step(df_nc.wl_um, net, label='Composite ($f=40$)')

plt.step(df_nc.wl_um, (1-ff)*smoothed_nocloud, label='Cloud-free')
plt.step(df_clouds.wl_um, ff*smoothed_cloudy, label='Cloudy')
plt.title(r'$T_{\mathrm{eff}}$ = 1000 K, $g=178$ m/s$^2$')
plt.legend(loc='best');
plt.xlim(2.240, 2.2600)